# Algoritmos de busqueda

La busqueda es una operación **escencial** que consiste en encontrar un elemento dentro de una colección de datos.

## Busqueda lineal

Consiste en buscar elemento por elemento hasta encontrar lo que se busca. Para eso se recorre cada elemento de la colección de tal manera que si el elemento es encontrado el algoritmo de detiene retornando el resultado, pero si no el elgoairmo continia retornando el indice.

Looping through each element

Element found
algorithm stops
returns the result
Element not found
algorithm continues


In [2]:
def linear_search(list, item):
    """Busca un ítem en una lista usando el algoritmo de búsqueda lineal. Este algoritmo revisa cada elemento en secuencia (de principio a fin)
    hasta que encuentra una coincidencia.

    Args:
        list (list): La lista (o secuencia) en la que se va a buscar.
        item (any): El valor que se desea encontrar.

    Returns:
        int: El índice de la *primera* ocurrencia del ítem en la lista.
             Retorna -1 si el ítem no se encuentra.
    """
    for i in range(len(list)):
        if list[i] == item:
            return i
    return -1

In [ ]:
def binary_search(sorted_list, item):
    """
    Busca un ítem en una lista "ordenada" usando el algoritmo de búsqueda binaria.

    Args:
        sorted_list (list): La lista (o secuencia) en la que se va a buscar.
                            Debe estar previamente ordenada.
        item (any): El valor que se desea encontrar.

    Returns:
        int: El índice del ítem en la lista.
             Retorna -1 si el ítem no se encuentra.
    """
    low = 0
    high = len(sorted_list) - 1
    while low <= high:
        mid = (low + high) // 2
        guess = sorted_list[mid]
        if guess == item:
            return mid
        if guess > item:
            high = mid - 1
        else:
            low = mid + 1
    return -1

## Refererencias

1. https://algs4.cs.princeton.edu/14analysis/
2. https://introcs.cs.princeton.edu/python/23recursion/
3. https://www.cs.princeton.edu/courses/archive/spring21/cos226/lectures/study/14AnalysisOfAlgorithms.html